In [1]:
# -*- coding:utf-8 -*-

from __future__ import print_function
from __future__ import division

import numpy as np
import pandas as pd
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
from tensorflow.python.layers.core import Dense
import sys
import os
import time
from tqdm import tqdm 

sys.path.append('..')
from data_process.sequence_example_helpers import get_padded_batch
from data_process.get_ids import get_vocab_map

In [2]:
W_encoder_embedding = np.load('../data/en_embeddings.npy')
W_decoder_embedding = np.load('../data/zh_embeddings.npy')
print('W_encoder_embedding.shape=', W_encoder_embedding.shape)
print('W_decoder_embedding.shape=', W_decoder_embedding.shape)


dict_en_word2id, dict_en_id2word = get_vocab_map('../data/vocab.en')
dict_zh_word2id, dict_zh_id2word = get_vocab_map('../data/vocab.zh')

GO_ID = dict_zh_word2id['<GO>']
EOS_ID = dict_zh_word2id['<EOS>']
EN_UNK_ID = dict_en_word2id['<UNK>']
EN_PAD_ID = dict_en_word2id['<PAD>']
print('EN_UNK_ID=', EN_UNK_ID)

W_encoder_embedding.shape= (87509, 256)
W_decoder_embedding.shape= (129939, 256)
vocab file ../data/vocab.en, vocab size = 87509
vocab file ../data/vocab.zh, vocab size = 129939
EN_UNK_ID= 1


In [3]:
# place_holder 部分

# lr = tf.placeholder(tf.float32, name='learning_rate')
# batch_size = tf.placeholder(tf.int64, [])
num_layers = 2
rnn_size = 256
start_lr = 1e-3
decay_steps = 10000
decay_rate = 0.9
global_step = tf.Variable(0, trainable=False, dtype=tf.int64)
lr = tf.train.exponential_decay(start_lr, global_step, decay_steps, decay_rate, staircase=True)
batch_size = 32

# 输入
with tf.variable_scope('inputs'): 
    inputs = tf.placeholder(tf.int32, [None, None], name='inputs')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    # 定义target序列最大长度（之后target_sequence_length和source_sequence_length会作为feed_dict的参数）
    target_sequence_length = tf.placeholder(tf.int32, (None,), name='target_sequence_length')
    source_sequence_length = tf.placeholder(tf.int32, (None,), name='source_sequence_length')

    max_target_sequence_length = tf.reduce_max(target_sequence_length, name='max_target_len')


In [4]:
# embeddings 
with tf.variable_scope('embeddings'):
    encoder_embeddings = tf.get_variable(name='encoder_embedding', shape=W_encoder_embedding.shape,
                                             initializer=tf.constant_initializer(W_encoder_embedding), trainable=True)
    decoder_embeddings = tf.get_variable(name='decoder_embedding', shape=W_decoder_embedding.shape,
                                             initializer=tf.constant_initializer(W_decoder_embedding), trainable=True)

# encoder
encoder_embed_input = tf.nn.embedding_lookup(encoder_embeddings, inputs)

# RNN cell
def get_lstm_cell(rnn_size):
    lstm_cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
    return lstm_cell

encoder_cell = tf.contrib.rnn.MultiRNNCell([get_lstm_cell(rnn_size) for _ in range(num_layers)])

encoder_output, encoder_state = tf.nn.dynamic_rnn(encoder_cell, encoder_embed_input, 
                                                  sequence_length=source_sequence_length, dtype=tf.float32)

# encoder_output
#   If time_major == False (default), this will be a `Tensor` shaped:
#     `[batch_size, max_time, cell.output_size]`.
#   If time_major == True, this will be a `Tensor` shaped:
#     `[max_time, batch_size, cell.output_size]`.
print(encoder_output) 
print(encoder_state)  # encoder_state 为最后一步的输出状态

Tensor("rnn/transpose:0", shape=(?, ?, 256), dtype=float32)
(LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_2:0' shape=(?, 256) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 256) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 256) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_5:0' shape=(?, 256) dtype=float32>))


In [5]:
# decoder
target_vocab_size = W_decoder_embedding.shape[0]
decoder_input = targets[:, :-1]  # 输入去掉最后的一个符号
decoder_embed_input = tf.nn.embedding_lookup(decoder_embeddings, decoder_input)
decoder_cell = tf.contrib.rnn.MultiRNNCell([get_lstm_cell(rnn_size) for _ in range(num_layers)])
output_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))

# 4. Training decoder
with tf.variable_scope("decode"):
    # 得到help对象
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=decoder_embed_input,
                                                        sequence_length=target_sequence_length,
                                                        time_major=False)
    # 构造decoder
    training_decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell,
                                                       training_helper,
                                                       encoder_state,
                                                       output_layer) 
    training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                   impute_finished=True,
                                                                   maximum_iterations=max_target_sequence_length)
# 5. Predicting decoder
# 与training共享参数
with tf.variable_scope("decode", reuse=True):
    # 创建一个常量tensor并复制为batch_size的大小
    start_tokens = tf.tile(tf.constant([GO_ID], dtype=tf.int32), [batch_size], name='start_tokens')
    predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(decoder_embeddings,
                                                            start_tokens,
                                                            EOS_ID)
    predicting_decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell,
                                                    predicting_helper,
                                                    encoder_state,
                                                    output_layer)
    #  returns: (final_outputs, final_state, final_sequence_lengths)  
    predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(predicting_decoder,
                                                        impute_finished=True,
                                                        maximum_iterations=max_target_sequence_length)
      
print(training_decoder_output)
print(training_decoder_output.rnn_output)  # [batch_size, max_time, target_vocab_size]，每步输出的类别概率
print(training_decoder_output.sample_id)   # [batch_size, max_time] ? 应该是每一步的预测 id 

BasicDecoderOutput(rnn_output=<tf.Tensor 'decode/decoder/transpose:0' shape=(?, ?, 129939) dtype=float32>, sample_id=<tf.Tensor 'decode/decoder/transpose_1:0' shape=(?, ?) dtype=int32>)
Tensor("decode/decoder/transpose:0", shape=(?, ?, 129939), dtype=float32)
Tensor("decode/decoder/transpose_1:0", shape=(?, ?), dtype=int32)


#### 损失函数以及优化器

In [6]:
# 损失函数
training_logits = tf.identity(training_decoder_output.rnn_output, 'logits')  # 训练的时候需要使用 logits 来计算损失
training_preds = tf.identity(training_decoder_output.sample_id, 'train_predictins')
predicting_logits = tf.identity(predicting_decoder_output.sample_id, name='predictions')    # 预测的时候只需要得到 id 结果。
# 返回 0.0, 1.0 的一个矩阵，标注 target_input 的每一位是否为 <PAD>
masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')   
with tf.name_scope("optimization"):
    # Loss function
    training_targets = targets[:, 1:]
    cost = tf.contrib.seq2seq.sequence_loss(
        training_logits,
        training_targets,   # 去掉前面的 <GO>
        masks)

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients, global_step=global_step)

    
saver = tf.train.Saver(max_to_keep=5)

print(training_logits)     # 每一步预测的概率分布
print(predicting_logits)   # 每一步预测的最大值对应的id

Tensor("logits:0", shape=(?, ?, 129939), dtype=float32)
Tensor("predictions:0", shape=(32, ?), dtype=int32)


In [7]:
# 读入 tfrecord 文件
from data_process.sequence_example_helpers import count_records

train_dataset_dir = '../data/train_tfrecords/'
tfrecord_files = os.listdir(train_dataset_dir)
tfrecord_file_names = [train_dataset_dir + file_name for file_name in tfrecord_files]
from_tokens, to_tokens, from_tokens_len, to_tokens_len = get_padded_batch(tfrecord_file_names, batch_size)
tf.train.start_queue_runners(sess=sess)

INFO:tensorflow:Counting records in ../data/train_tfrecords/train-14-of-32.tfrecord.
INFO:tensorflow:Number of records is at least 1000.
INFO:tensorflow:[<tf.Tensor 'random_shuffle_queue_Dequeue:0' shape=(?,) dtype=int64>, <tf.Tensor 'random_shuffle_queue_Dequeue:1' shape=(?,) dtype=int64>, <tf.Tensor 'random_shuffle_queue_Dequeue:2' shape=() dtype=int64>, <tf.Tensor 'random_shuffle_queue_Dequeue:3' shape=() dtype=int64>]


[<Thread(Thread-19, started daemon 140348491589376)>,
 <Thread(Thread-20, started daemon 140348482934528)>,
 <Thread(Thread-21, started daemon 140348474541824)>,
 <Thread(Thread-22, started daemon 140348078192384)>,
 <Thread(Thread-23, started daemon 140348069799680)>]

In [ ]:
# 测试 get_batch
input_tensors = [from_tokens, to_tokens, from_tokens_len, to_tokens_len]
batch_inputs, batch_targets, batch_source_sequence_length, batch_target_sequence_length = sess.run(input_tensors)
print(batch_inputs.shape)
print(batch_targets.shape)
print(batch_source_sequence_length.shape)
print(batch_target_sequence_length.shape)

(32, 42)
(32, 43)
(32,)
(32,)


### 训练模型

In [ ]:
max_epoch = 5
_lr = 1e-3
sess.run(tf.global_variables_initializer())

n_tr_batches = 10000

for epoch in xrange(max_epoch):
    batch_indexs = np.random.permutation(n_tr_batches)  # shuffle the training data
    _costs = 0.0
    time0 = time.time()
    for batch in xrange(n_tr_batches):
        _global_step = sess.run(global_step)
        # training
        batch_id = batch_indexs[batch]
        batch_inputs, batch_targets, batch_source_sequence_length, batch_target_sequence_length = sess.run(input_tensors)
        train_fetches = [cost, train_op]
        feed_dict = {inputs: batch_inputs, targets: batch_targets, source_sequence_length: batch_source_sequence_length, 
                    target_sequence_length: batch_target_sequence_length, lr:_lr}
        _cost, _ = sess.run(train_fetches, feed_dict)  # the cost is the mean cost of one batch
        _costs += _cost
        sys.stdout.write('\r%d/%d, cost = %g, passed %gs' % (batch, n_tr_batches, _cost, time.time() - time0))
        sys.stdout.flush()
    mean_cost = _costs / n_tr_batches
    print('\t train cost = %g, time cost %gs ' % (mean_cost, time.time() - time0))
    save_path = saver.save(sess, '../ckpt/base_seq2seq.ckpt', global_step=sess.run(global_step))
    print(save_path)

# 19999/20000, cost = 4.46718, passed 11972.7s	 train cost = 4.55279, time cost 11972.7s 
# ../ckpt/base_seq2seq.ckpt-20000
# 1491/20000, cost = 3.56721, passed 882.626s

3668/10000, cost = 4.93511, passed 2122.07s

## 预测

### 导入模型

In [ ]:
ckpt_path = '../ckpt/'
saver.restore(sess, tf.train.latest_checkpoint(ckpt_path))

In [ ]:
def source_to_seq(sentence, max_len=150):
    """ 
    Args: 
        sentence: input English sentence, i.e. Eg: Nice to meet you.
        max_len: the limited length of the input sentence.
    Returns:
        input_ids: The id sequence of the English sentence.
        input_seq_len: The words number for the input sentence.
    """
    from nltk import word_tokenize
    words = word_tokenize(sentence.lower().decode('utf-8').strip())
    input_ids = [dict_en_word2id.get(word, EN_UNK_ID) for word in words]
    input_seq_len = len(input_ids)
    if input_seq_len > max_len:
        input_ids = input_ids[:max_len]
        input_seq_len = max_len
    return input_ids, input_seq_len

def seq_to_target(ids):
    """decode the target id to Chinese sentence."""
    words = [dict_zh_id2word[each_id] for each_id in ids]
    sentence = ' / '.join(words)
    return sentence


print(source_to_seq('Dear me, that is a long time not seeing you!'))
print(seq_to_target([1,123, 32,443]))

In [ ]:
def translate(sentence):
    input_ids, input_seq_len = source_to_seq(sentence)
    input_data = np.asarray(input_ids * batch_size).reshape([batch_size, -1])
    input_seq_len = np.asarray([input_seq_len] * batch_size).reshape([batch_size])
    fetch = [predicting_logits]
    # decode 最长长度默认使用输入的两倍长度。
    feed_dict = {inputs: input_data, source_sequence_length: input_seq_len, target_sequence_length: input_seq_len*2}  # 预测两倍长度
    answer_logits = sess.run(fetch, feed_dict=feed_dict)[0][0]
    zh_sentence = seq_to_target(answer_logits)
    return zh_sentence, len(answer_logits)

In [ ]:
en_sent1 = 'Hi, rabbit. Long time no see! How is it going?'
en_sent2 = 'Good morning, every one.'
zh_sent, sent_len = translate(en_sent1)
print('zh_sent len = %d' % sent_len)
print(zh_sent)

zh_sent, sent_len = translate(en_sent2)
print('zh_sent len = %d' % sent_len)
print(zh_sent)